### Configuration of the environment

In [1]:
%tensorflow_version 2.x
!pip3 install --upgrade pip
!pip install -qU t5


import functools
import os
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds

import t5

#Set the base dir(Google cloud bucket)
BASE_DIR = "gs://bucket_block_completion" 

if not BASE_DIR or BASE_DIR == "gs://":
  raise ValueError("You must enter a BASE_DIR.")
ON_CLOUD = True

!pip install tensorflow-gcs-config

if ON_CLOUD:
  import tensorflow_gcs_config
  from google.colab import auth
  # Set credentials for GCS reading/writing from Colab and TPU.
  TPU_TOPOLOGY = "2x2"
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    TPU_ADDRESS = tpu.get_master()
    print('Running on TPU:', TPU_ADDRESS)
  except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
  auth.authenticate_user()
  tf.config.experimental_connect_to_host(TPU_ADDRESS)
  tensorflow_gcs_config.configure_gcs_from_colab_auth()

tf.disable_v2_behavior()

# Improve logging.
from contextlib import contextmanager
import logging as py_logging

if ON_CLOUD:
  tf.get_logger().propagate = False
  py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)

     |████████████████████████████████| 1.7 MB 5.4 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     |████████████████████████████████| 153 kB 5.3 MB/s            
     |████████████████████████████████| 1.2 MB 30.4 MB/s            
     |████████████████████████████████| 90 kB 11.0 MB/s            
     |████████████████████████████████| 366 kB 81.7 MB/s            
     |████████████████████████████████| 4.0 MB 75.0 MB/s            
     |████████████████████████████████| 4.9 MB 64.4 MB/s            
     |████████████████████████████████| 3.1 MB 50.8 MB/s            
     |████████████████████████████████| 286 kB 83.5 MB/s            
     |████████████████████████████████| 59 kB 5.7 MB/s             
     |████████████████████████████████| 895 kB 75.5 MB/s            
     |████████████████████████████████| 596 kB 80.9 MB/s            
     |████████████████████████████████| 3.3 

Instructions for updating:
non-resource variables are not supported in the long term


### Loading of the tsv files
We loaded the tsv files, please be sure to upload them on the bucket and to copy the correct path

In [2]:

nq_tsv_path = {
    "train":      'gs://bucket_block_completion/dataset_08_07/train.tsv',
    "validation": 'gs://bucket_block_completion/dataset_08_07/eval.tsv',
}

num_nq_examples = dict(train=1197310, validation=15783)

### Preprocess of the dataset
In this step we preprocess the dataset.  
You have to change the path to vocab files (*vocab_model_path* and *vocab_path*)
We're going to preprocess all the tsv file so that T5 can use them for HP tuning.

In [3]:
from t5.data import postprocessors as t5_postprocessors
from t5.seqio import Feature,SentencePieceVocabulary


# # Set the path of sentencepiece model and vocab files
# # Must be the same used for the pre-trained phase
vocab_model_path = 'gs://bucket_block_completion/code.model'
vocab_path = 'gs://bucket_block_completion/code.vocab'


TaskRegistry = t5.data.TaskRegistry
TfdsTask = t5.data.TfdsTask


def get_default_vocabulary():
  return SentencePieceVocabulary(vocab_model_path, 100)

DEFAULT_OUTPUT_FEATURES = {
    "inputs": Feature(
        vocabulary=get_default_vocabulary(), add_eos=True, required=False),

    "targets": Feature(
        vocabulary=get_default_vocabulary(), add_eos=True)
}

## If you have more than one task you have to use the prefix, otherwise it is not mandatory

In [4]:
prefix=''

def nq_dataset_task(split, shuffle_files=True):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.

  ds = tf.data.TextLineDataset(nq_tsv_path[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw train examples...")
for ex in tfds.as_numpy(nq_dataset_task("train").take(5)):
    print(ex)


def preprocessing(ds):
  
  def to_inputs_and_targets(ex):
        inputs = tf.strings.join([prefix + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label}
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
      
t5.data.TaskRegistry.remove('finetuning')
t5.data.TaskRegistry.add(
    "finetuning",
    dataset_fn=nq_dataset_task,
    splits=["train","validation"],
    text_preprocessor=preprocessing,
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy]
)

A few raw train examples...
{'input': b'public static void saveBitmapToDevice(Bitmap btmp, String filePath, String imageName) { File file = new File (filePath, imageName); if (file.exists ()) file.delete (); try <extra_id_0> catch (Exception e) { e.printStackTrace(); } }', 'output': b'{ FileOutputStream out = new FileOutputStream(file); btmp.compress(Bitmap.CompressFormat.JPEG, 90, out); out.flush(); out.close(); }'}
{'input': b'@Override public Dampening updateGroupDampening(String tenantId, Dampening groupDampening) throws Exception { if (isEmpty(tenantId)) { throw new IllegalArgumentException("TenantId must be not null"); } if (isEmpty(groupDampening)) { throw new IllegalArgumentException("DampeningId and TriggerId must be not null"); } try { deferNotifications(); checkTenantId(tenantId, groupDampening); String groupId = groupDampening.getTriggerId(); Trigger groupTrigger = getTrigger(tenantId, groupId); if (!groupTrigger.isGroup()) { throw new IllegalArgumentException( "Trigger [" 

In [5]:
nq_task = t5.data.TaskRegistry.get("finetuning")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 512, "targets": 512})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)

A few preprocessed training examples...
{'inputs_pretokenized': b'@Nonnull public QValue getQValueOfLanguage (@Nonnull final String sLanguage) { ValueEnforcer.notNull (sLanguage, "Language"); QValue aQuality = m_aMap.get (_unify (sLanguage)); if (aQuality == null) <extra_id_0> return aQuality; }', 'inputs': array([   19,   790,    12,     3,     2,   106,    41,     2,   106,
         432,  1354,     3,   261,   790,    44,    26,     3,    22,
        1354,     8,     7,     3,   106, 30104,     4,  2623,    17,
          22,  1354,     9,    32,  1354,    46,     3,     2,   106,
         107,     2,  3145,    11,    54,    15,   184,   100,     4,
          33,    17,    15,  1284,  3647,    17,    22,  1354,    79,
          21,    17,   184,     2,  3145,    40,    30,     8, 32099,
          14,   107,     2,  3145,    13,     6,     1], dtype=int32), 'targets_pretokenized': b'{ aQuality = m_aMap.get (AcceptLanguageHandler.ANY_LANGUAGE); if (aQuality == null) { return QValue.MIN_

### Hyper Parameter tuning
You can run the HP tuning using the following cells.  
Please set the correct path of the variable *MODEL_DIR* (the path to save the pretrained model in), *PATH_GIN_FILE* (the gin file configuration for this HP tuning) and *PRETRAINED_DIR* (the folder that contains the pretrained model).  
**Keep attention** to change the *pretrained_model_dir* in finetune step (if you are starting the HP tuning from scratch you have to set the value *PRETRAINED_DIR*, if you are restarting the HP tuning from a previous saved checkpoint you have to set the value *MODEL_DIR*)

In [ ]:
## this is useful if you have more than one task
# def _rate_num_input_examples(task):
#   if "train" in task.splits:
#     return float(task.num_input_examples("train"))
#   elif "validation" in task.splits:
#     return float(task.num_input_examples("validation"))
#   else:
#     raise ValueError("Task %s does not have a train or validation split." % (task.name))


# t5.data.MixtureRegistry.remove("all_tasks")
# t5.data.MixtureRegistry.add(
#     "all_tasks",
#     ["java_construct", "java_token", "java_block", "android_construct", "android_token", "android_block"],
#     default_rate=_rate_num_input_examples
#      #default_rate=1.0
# )

SLANTED

In [ ]:
from t5 import models

MODEL_SIZE = "small" 

# Set the folder where the checkpoints and all the others information will be writed
MODEL_DIR = 'gs://bucket_block_completion/HP_TUNING/slanted/model'

# Specify the pre-trained dir which must contain the pre-trained models, the operative_config.gin file and the checkpoint file as well
PRETRAINED_DIR='gs://bucket_block_completion/pretrained_model'


model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 256, 16),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]

tf.io.gfile.makedirs(MODEL_DIR)

model = models.mtf_model.MtfModel(
    model_dir=MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    sequence_length={"inputs": 512, "targets": 512},
    save_checkpoints_steps=5000,
    keep_checkpoint_max=keep_checkpoint_max if ON_CLOUD else None,
    iterations_per_loop=100,
)

In [ ]:
# we used model.predict function (setting beam_size)

vocabulary_predict=get_default_vocabulary()

model.predict(input_file='gs://bucket_block_completion/HP_TUNING/slanted/model/validation_eval/finetuning_inputs', output_file='gs://bucket_block_completion/HP_TUNING/slanted/model/validation_eval/predictions.txt',
              checkpoint_steps=250000, beam_size=1, temperature=0.0, keep_top_k=-1, vocabulary=vocabulary_predict)

INFO:root:system_path_file_exists:gs://bucket_block_completion/HP_TUNING/slanted/model/operative_config.gin
ERROR:root:Path not found: gs://bucket_block_completion/HP_TUNING/slanted/model/operative_config.gin


INFO:tensorflow:Using config: {'_model_dir': 'gs://bucket_block_completion/HP_TUNING/slanted/model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.126.119.98:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.126.119.98:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.126.119.98:8470', '_evaluation_master': 'grpc://10.126

CONSTANT

In [ ]:
from t5 import models

MODEL_SIZE = "small" 

# Set the folder where the checkpoints and all the others information will be writed
MODEL_DIR = 'gs://bucket_block_completion/HP_TUNING/constant/model'

# Specify the pre-trained dir which must contain the pre-trained models, the operative_config.gin file and the checkpoint file as well
PRETRAINED_DIR='gs://bucket_block_completion/pretrained_model'


model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 256, 16),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]

tf.io.gfile.makedirs(MODEL_DIR)

model = models.mtf_model.MtfModel(
    model_dir=MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    sequence_length={"inputs": 512, "targets": 512},
    save_checkpoints_steps=5000,
    keep_checkpoint_max=keep_checkpoint_max if ON_CLOUD else None,
    iterations_per_loop=100,
)

In [ ]:
# we used model.predict function (setting beam_size)

vocabulary_predict=get_default_vocabulary()

model.predict(input_file='gs://bucket_block_completion/HP_TUNING/constant/model/validation_eval/finetuning_inputs', output_file='gs://bucket_block_completion/HP_TUNING/constant/model/validation_eval/predictions.txt',
              checkpoint_steps=250000, beam_size=1, temperature=0.0, keep_top_k=-1, vocabulary=vocabulary_predict)

INFO:root:system_path_file_exists:gs://bucket_block_completion/HP_TUNING/constant/model/operative_config.gin
ERROR:root:Path not found: gs://bucket_block_completion/HP_TUNING/constant/model/operative_config.gin


INFO:tensorflow:Using config: {'_model_dir': 'gs://bucket_block_completion/HP_TUNING/constant/model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.126.119.98:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.126.119.98:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.126.119.98:8470', '_evaluation_master': 'grpc://10.12

ISR

In [ ]:
from t5 import models

MODEL_SIZE = "small" 

# Set the folder where the checkpoints and all the others information will be writed
MODEL_DIR = 'gs://bucket_block_completion/HP_TUNING/ISR/model'

# Specify the pre-trained dir which must contain the pre-trained models, the operative_config.gin file and the checkpoint file as well
PRETRAINED_DIR='gs://bucket_block_completion/pretrained_model'


model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 256, 16),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]

tf.io.gfile.makedirs(MODEL_DIR)

model = models.mtf_model.MtfModel(
    model_dir=MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    sequence_length={"inputs": 512, "targets": 512},
    save_checkpoints_steps=5000,
    keep_checkpoint_max=keep_checkpoint_max if ON_CLOUD else None,
    iterations_per_loop=100,
)

In [ ]:
# we used model.predict function (setting beam_size)

vocabulary_predict=get_default_vocabulary()

model.predict(input_file='gs://bucket_block_completion/HP_TUNING/ISR/model/validation_eval/finetuning_inputs', output_file='gs://bucket_block_completion/HP_TUNING/ISR/model/validation_eval/predictions.txt',
              checkpoint_steps=250000, beam_size=1, temperature=0.0, keep_top_k=-1, vocabulary=vocabulary_predict)

INFO:root:system_path_file_exists:gs://bucket_block_completion/HP_TUNING/ISR/model/operative_config.gin
ERROR:root:Path not found: gs://bucket_block_completion/HP_TUNING/ISR/model/operative_config.gin


INFO:tensorflow:Using config: {'_model_dir': 'gs://bucket_block_completion/HP_TUNING/ISR/model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.126.119.98:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.126.119.98:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.126.119.98:8470', '_evaluation_master': 'grpc://10.126.119

POLYNOMIAL

In [6]:
from t5 import models

MODEL_SIZE = "small" 

# Set the folder where the checkpoints and all the others information will be writed
MODEL_DIR = 'gs://bucket_block_completion/HP_TUNING/polynomial/model'

# Specify the pre-trained dir which must contain the pre-trained models, the operative_config.gin file and the checkpoint file as well
PRETRAINED_DIR='gs://bucket_block_completion/pretrained_model'


model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 256, 16),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]

tf.io.gfile.makedirs(MODEL_DIR)

model = models.mtf_model.MtfModel(
    model_dir=MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    sequence_length={"inputs": 512, "targets": 512},
    save_checkpoints_steps=5000,
    keep_checkpoint_max=keep_checkpoint_max if ON_CLOUD else None,
    iterations_per_loop=100,
)

In [7]:
# we used model.predict function (setting beam_size)

vocabulary_predict=get_default_vocabulary()

model.predict(input_file='gs://bucket_block_completion/HP_TUNING/polynomial/model/validation_eval/finetuning_inputs', output_file='gs://bucket_block_completion/HP_TUNING/polynomial/model/validation_eval/predictions.txt',
              checkpoint_steps=250000, beam_size=1, temperature=0.0, keep_top_k=-1, vocabulary=vocabulary_predict)

INFO:root:system_path_file_exists:gs://bucket_block_completion/HP_TUNING/polynomial/model/operative_config.gin
ERROR:root:Path not found: gs://bucket_block_completion/HP_TUNING/polynomial/model/operative_config.gin


INFO:tensorflow:Using config: {'_model_dir': 'gs://bucket_block_completion/HP_TUNING/polynomial/model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.12.62.226:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.12.62.226:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.12.62.226:8470', '_evaluation_master': 'grpc://10.12.